In [1]:
#! pip install pandas

import re
import pandas as pd

In [2]:
def parse_log(filename, limit_n_rows=None):
    statistics_regexp = re.compile(r".*<<\{(\d+),\s+(\d+.\d*),\s(\d+)}>>.*")
    statistics = {'id':[], 'value':[], 'nano_delta':[]}
    
    with open(f'./resources/{filename}', 'r') as log_file:
        line = log_file.readline()
        while line:
            groups = re.search(statistics_regexp, line)
            if groups:
                _id, value, nano_delta = groups.groups()
                statistics['id'].append(_id)
                statistics['value'].append(value)
                statistics['nano_delta'].append(nano_delta)
            line = log_file.readline()
    
    df_statistics = pd.DataFrame.from_dict(statistics).astype({'id': int, 'value': float, 'nano_delta': int})
    df_statistics["micro_delta"] = df_statistics["nano_delta"] / 1e3
    
    if limit_n_rows:
        df_statistics = df_statistics.head(limit_n_rows) 
    
    return df_statistics

In [3]:
runs = {}

In [4]:
runs["1m runs, 2 VMs, UDP"] = parse_log("run._two_sep_vms.log", 1_000_000)

In [5]:
runs["1m runs, 1 VMs, UDP"] = parse_log("run.one_vm_udp.log", 1_000_000)

In [6]:
runs["1m runs, 1 VMs, IPC"] = parse_log("run.one_vm_ipc.log", 1_000_000)

In [7]:
overall_stats = {'name':[], 'alerts_received':[], 'fastest':[], '0.5q':[],'0.9q':[],'0.95q':[],'0.99q':[],'slowest':[], 'average':[]}
for _name, stats in runs.items():
    _df = stats[["micro_delta"]]
    overall_stats['name'].append(_name)
    overall_stats['alerts_received'].append(len(_df))
    overall_stats['fastest'].append(_df.min().iloc[0])
    overall_stats['0.5q'].append(_df.quantile(0.5).iloc[0])
    overall_stats['0.9q'].append(_df.quantile(0.9).iloc[0])
    overall_stats['0.95q'].append(_df.quantile(0.95).iloc[0])
    overall_stats['0.99q'].append(_df.quantile(0.99).iloc[0])
    overall_stats['slowest'].append(_df.max().iloc[0])
    overall_stats['average'].append(_df.mean().iloc[0])

df_overall_stats = pd.DataFrame.from_dict(overall_stats)
df_overall_stats

,name,alerts_received,fastest,0.5q,0.9q,0.95q,0.99q,slowest,average
0,"1m runs, 2 VMs, UDP",1000000,220.843,1402.4430,2201.4477,2418.18520,2879.53944,23480.850,1470.601292
1,"1m runs, 1 VMs, UDP",1000000,98.370,1276.2855,2274.5690,2511.18420,3201.68512,23693.544,1402.269281
2,"1m runs, 1 VMs, IPC",1000000,7.538,520.0455,991.1941,1046.27405,1096.41008,11327.798,532.653113
